# Glacier Velocity Mapping using Template Matching

Franz J Meyer; University of Alaska Fairbanks

Forrest Williams; Alaska Satellite Facility

This lab will let you exercise template matching techniques for the application of glacier velocity mapping. We will initially perform template matching using cross-correlation techniques on a pair of Sentinel-2 optical images covering an are of south east Alaska that includes Malaspina and Hubbard Glaciers. Subsequently, we will work with a large number of Sentinel-1 image pairs processed using the AutoRIFT algorithm covering the same area. This larger set of images will help understand recent changes in glacier velocity at these glaciers.

It will introduce the following data analysis concepts:

- Using template matching for the application of glacier velocity mapping
- How to order template matching-based offset maps using the AutoRIFT algorithm
- Analysis of glacier velocity changes at Maimport

## 0. Importing Relevant Python Packages, Environment Setup, and Defining Helper Functions

In [ ]:
%matplotlib inline
from autoRIFT import autoRIFT
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.patches as patches
import zipfile
import pandas as pd

In [ ]:
# Creating target folder
path = f"{os.getcwd()}/data"
if not os.path.exists(path):
    os.makedirs(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
def load_gdal(infile, band=1, subset=[0,0,None,None]):
    ds = gdal.Open(infile, gdal.GA_ReadOnly)
    
    if not band:
        data = ds.ReadAsArray(*subset)
    else:
        data = ds.GetRasterBand(band).ReadAsArray(*subset)
        
    nodata = ds.GetRasterBand(1).GetNoDataValue()
    mask = data == nodata
    data = np.ma.array(data, mask=mask, fill_value=0)
    projection = ds.GetProjection()
    transform = ds.GetGeoTransform()
    del ds
    return data, transform, projection

## 1. Introduction to the Workflow We Will Use

### 1.1 The Basic Template Matching Approach Used Here

`AutoRIFT` is a template matching python library that is used in production by the [ITS_LIVE project](https://its-live.jpl.nasa.gov) to track glacier velocities across the globe. It determines offsets at a user-specified grid spacing by comparing a square patch of pixels (a "chip") from an earlier image to the pixels in a larger square patch in a later image using an `openCV`-based **Normalized Cross Correlation** method (see figure below). The Normalized Cross Correlation analysis returns a correlation surface at integer pixel offsets between the two image chips which can be used to estimated sub-pixel offsets between the two images. These offsets are determined by finding the peak of the spline of the correlation surface in the vicinity of the highest integer correlation peak.


## 1.2 Benefits of the AutoRIFT Algorithm

<img style="padding: 0px 0px 0px 10px" src="Figs/TemplateMatching.JPG" width="500" align="right" /> Because `AutoRIFT` takes `numpy` array as inputs, it can be used images of basically any format, with the caveat that the images must have the same resolution and extent. In situations where this is not case, external utilities like `GDAL` can be used to transform the input data and convert it to `numpy`. `AutoRIFT` also outputs a `numpy` array with offsets at a user-specified grid resolution in the same projection as the original images. These arrays can then be written to a variety of geospatial image formats via a external tools (i.e. `GDAl`).

`AutoRIFT` is computationally fast because of the use of `numpy` and the `openCV` library, and so can process an image pair in minutes or tens of minutes depending on the image sizes and requested grid spacing and maximum offset specified for the search. This process can be sped up by using land and ocean masks to limit search distances off of the ice and also by using reference ice flow speed maps to set search distances larger in fast flowing areas and smaller in slow flowing areas, but for simplest uses these are not applied.

# 2. Exercising PyCorr on a Sentinel-2 Image Pair over Malaspina Glacier

## 2.1 Accessing the Sentinel-2 Data

We will use a pair of Sentinel-2 images acquired in May 2020 to demonstrate how `AutoRIFT` works and which parameters it needs for creating an offset field. You can replicate this run by replacing the image pairs used below.

We first get two Sentinel 2 images from the [AWS S2 Level2A CloudOptimizedGeotiff (COG) public data archive](https://registry.opendata.aws/sentinel-2-l2a-cogs/)

In [ ]:
!curl https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/7/V/EG/2020/5/S2B_7VEG_20200512_0_L2A/B08.tif --output S2B_7VEG_20200512_0_L2A_B08.tif
!curl https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/7/V/EG/2020/5/S2A_7VEG_20200527_0_L2A/B08.tif --output S2A_7VEG_20200527_0_L2A_B08.tif

<img style="padding: 0px 0px 0px 10px" src="Figs/Malaspina-Hubbard.jpg" width="500" align="right" />This image pair we will be analyzing in this part of the notebook covers a part of south-east Alaska including Malaspina and Hubbard glacier.

<b>Malaspina glacier</b> is the largest piedmont glacier in the world. Situated at the head of the Alaska Panhandle, it is about 65 km (40 mi) wide and 45 km (28 mi) long, with an area of some 3,900 $km^2$ (1,500 $mi^2$). It is named in honor of Alessandro Malaspina, a Tuscan explorer in the service of the Spanish Navy, who visited the region in 1791. Malaspina has been undergoing a surge that started in around May 2020 and seems to have ended near the end of 2022.

<b>Hubbard Glacier</b> is a glacier located in <b><a href="https://en.wikipedia.org/wiki/Wrangell%E2%80%93St._Elias_National_Park_and_Preserve" target="_blank">Wrangell–St. Elias National Park and Preserve</a></b> in eastern Alaska and Kluane National Park and Reserve in Yukon, Canada, and named after Gardiner Hubbard. The longest source for Hubbard Glacier originates 122 kilometers (76 mi) from its snout and is located at about 61°00′N 140°09′W, approximately 8 kilometers (5 mi) west of Mount Walsh with an elevation around 11,000 feet (3,400 m). A shorter tributary glacier begins at the easternmost summit on the Mount Logan ridge at about 18,300 feet (5,600 m) at about 60°35′0″N 140°22′40″W.

Before it reaches the sea, Hubbard is joined by the Valerie Glacier to the west, which, through forward surges of its own ice, has contributed to the advance of the ice flow that experts believe will eventually dam the Russell Fjord from Disenchantment Bay waters.

## 2.2 Running AutoRIFT to Perform Template Matching-based Glacier Velocity Mapping

We now will run the `AutoRIFT` algorithm to perform template matching on the Sentinel-2 image pair. The first thing we'll do is define a helper function. This function takes AutoRIFT's main processing parameters as inputs, but abstracts away some of the nitty-gritty details so that you can perform analyses quickly.

In [ ]:
def prep_autorift(reference, secondary, step=16, min_window=32, max_window=128, min_distance=0, max_distance=10):
    # intizialize
    obj = autoRIFT()

    # set imagery
    obj.I1 = reference.astype(np.float32)
    obj.I2 = secondary.astype(np.float32)

    # Sampling
    obj.SkipSampleX = step
    obj.SkipSampleY = step

    # Chip size
    obj.ChipSizeMinX = min_window
    obj.ChipSizeMaxX = max_window
    obj.ChipSize0X = float(16)
    obj.ScaleChipSizeY = int(1)

    # Search Area
    obj.minSearch = min_distance
    obj.SearchLimitX = max_distance
    obj.SearchLimitY = max_distance

    # Result grid
    m, n = reference.shape
    xVals = np.arange(obj.SkipSampleX, n - obj.SkipSampleX, obj.SkipSampleX)
    yVals = np.arange(obj.SkipSampleY, m - obj.SkipSampleY, obj.SkipSampleY)
    ncol = int(xVals.shape[0])
    nrow = int(yVals.shape[0])
    xGrid = np.repeat(np.reshape(xVals, (1, ncol)), nrow, axis=0)
    yGrid = np.repeat(np.reshape(yVals, (nrow, 1)), ncol, axis=1)
    obj.xGrid = xGrid
    obj.yGrid = yGrid

    return obj

Next we'll read in the input data and subset it to our area of interest.

In [ ]:
bounds = np.s_[1000:7500, 0:5000]
ref, _, _ = load_gdal('S2B_7VEG_20200512_0_L2A_B08.tif')
ref = ref[bounds]
sec, _, _ = load_gdal('S2A_7VEG_20200527_0_L2A_B08.tif')
sec = sec[bounds]

Let's take a look at our input images. These are grayscale panchromatic images, but you can clearly see the Malaspina Glacier in both.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

img_color_opts = {'vmin': 0, 'vmax': 12000, 'cmap': 'Greys_r'}

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

dx_plot = ax1.imshow(ref, **img_color_opts)
ax1.set(title='May 12, 2020', aspect='equal', xticks=[], yticks=[])

dy_plot = ax2.imshow(sec, **img_color_opts)
ax2.set(title='May 27, 2020', aspect='equal', xticks=[], yticks=[])

f.tight_layout()

Now we'll actually run AutoRIFT. We first prep the AutoRIFT processing object, then we make sure the inputs have a uniform data type, next we filter the input data using a high-pass filter to reduce the noise in our analysis, then finally we run the template matching.

You'll see some warnings pop up when you run AutoRIFT. Don't mind those.

In [ ]:
# Run autoRIFT
step = 16
result = prep_autorift(ref, sec, step=step)
result.uniform_data_type()
result.preprocess_filt_hps()
result.runAutorift()

And done!
Let's a take a look at the results...

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

vel_color_opts = {'vmin': -1000, 'vmax': 1000, 'cmap': 'RdBu'}

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

dx_plot = ax1.imshow(result.Dx * step * 10, **vel_color_opts)
ax1.set(title='West-to-East Movement (m) May 12-27, 2020', aspect='equal', xticks=[], yticks=[])

dy_plot = ax2.imshow(result.Dy * step * 10, **vel_color_opts)
ax2.set(title='North-to-South Movement (m) May 12-27, 2020', aspect='equal', xticks=[], yticks=[])

f.colorbar(dx_plot, ax=ax1)
f.colorbar(dy_plot, ax=ax2)
f.tight_layout()

The left image shows the motion AutoRIFT detected in the West-to-East (dX) direction, and the right image shows the detected motion in the South-to-North (dY) direction. In the fastest moving parts of the glacier, it looks like the glacier has moved over 1 km (1000 m) in less than a month!

# 3. Stack of Velocity Maps over Malaspina Glacier Generated using AutoRIFT

We have seen in the previous example how effective template matching can be for the application of glacier velocity mapping. BUT, we've also seen that the process of template matching is time-consuming. Therefore, **offloading the process of ice velocities mapping to an automatic service** is appealing, especially when one is interested in mapping glacier motion across a longer time span.

In partnership with the ITS_LIVE project, the Alaska Satellite Facility has recently made the template-tracking based AutoRIFT processor available through its search interface Vertex. To order AutoRIFT data, follow the following general workflow:
1. Go to Vertex at [https://search.asf.alaska.edu/](https://search.asf.alaska.edu/)
2. Search for Sentinel-1 Single Look Complex (SLC) images over your area of interest
3. Use the SBAS search to create image pairs for AutoRIFT processing.
4. Click on the "custom processing icon (three nested squares) to submit your jobs

## 3.1 Retrieving AutoRIFT Stack over Malaspina Glacier

Using the approach described above, I have ordered 1-year worth of AutoRIFT velocity maps from <b><a href="https://search.asf.alaska.edu/" target="_blank">ASF</a></b>. In total, I have collected 70 glacier velocity estimates over this time period, using 12-day and 24-day Sentinel-1 SAR pairs.

I have prepared these data sets and deposited them for you in an AWS S3 storage bucket. In a first step, we will <b>download these data and unzip them</b> into a folder called ```lab_6_data```.

In [ ]:
!aws --region=us-west-2 --no-sign-request s3 cp s3://asf-jupyter-data-west/MalaspinaAutoRIFT.zip MalaspinaAutoRIFT.zip

In [ ]:
with zipfile.ZipFile('MalaspinaAutoRIFT.zip', 'r') as zip_obj:
   zip_obj.extractall()

## 3.2 Create VRT And Explore Data Dimensions

We first <b>define two python functions</b> to identify all of our ```AutoRIFT``` Tiff files and to extract the image dates for these files. Because I am lazy, I am using the date of the reference image here. A more sophisticated approach could be to use the mean date between reference and secondary image.

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

def get_dates(pths):
    dates = []
    for p in pths:
        for name_chunk in p.split('/')[-1].split('_'):
            nums = list(range(48, 58))
            if len(name_chunk) == 15 and ord(name_chunk[0]) in nums:
                date = name_chunk.split('T')[0]
                dates.append(date)
                break
    dates.sort()
    return dates

Now we <b>use these functions</b> to identify our GeoTIFFs and extract associated image dates.

In [ ]:
paths = "*S1*.tif*"
tiff_paths = get_tiff_paths(paths)
dates=get_dates(tiff_paths)

We also <b>create a Pandas time index</b> fpr our data and print the image dates. You see we have 70 image pairs in our stack with reference image times between Jan 09, 2021 and March 29, 2022

In [ ]:
time_index = pd.DatetimeIndex(dates)

for jacqdate, acqdate in enumerate(time_index):
    print('{:4d} {}'.format(jacqdate, acqdate.date()),end=' ')
    if (jacqdate % 5 == 4):
        print()

Now we <b>read the data</b> and mask out values associated with bad matches.

In [ ]:
rasterstack_masked = np.ma.stack([load_gdal(x)[0] for x in tiff_paths])
rasterstack_masked = np.rot90(rasterstack_masked, k=1, axes=(1,2))

## 3.3 Visualize Mean Glacier Velocity and Enable Extracting of Pixel-by-Pixel Velocity Time Series Extraction

We will now visualize an average velocity image in a way that we can move our mouse over the image and visualize the line/sample image coordinates. This will help us create time-series information for the most interesting image locations.

In [ ]:
temporal_mean = np.nanmean(rasterstack_masked, axis=0)

To do so, we first **create some helper functions:**

In [ ]:
import warnings

warnings.filterwarnings('ignore')

class pixelPicker:
    def __init__(self, image, width, height):
        self.x = None
        self.y = None
        self.fig = plt.figure(figsize=(width, height))
        self.ax = self.fig.add_subplot(111, visible=False)
        self.rect = patches.Rectangle(
            (0.0, 0.0), width, height,
            fill=False, clip_on=False, visible=False
        )

        self.rect_patch = self.ax.add_patch(self.rect)
        self.cid = self.rect_patch.figure.canvas.mpl_connect('button_press_event',self)
        self.image = image
        self.plot = self.gray_plot(self.image, fig=self.fig, return_ax=True)
        self.plot.set_title('Select a Point of Interest')


    def gray_plot(self, image, vmin=None, vmax=None, fig=None, return_ax=False):
        '''
        Plots an image in grayscale.
        Parameters:
        - image: 2D array of raster values
        - vmin: Minimum value for colormap
        - vmax: Maximum value for colormap
        - return_ax: Option to return plot axis
        '''
        if vmin is None:
            vmin = np.nanpercentile(self.image, 10)
        if vmax is None:
            vmax = np.nanpercentile(self.image, 99)
        if fig is None:
           my_fig = plt.figure()
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        pos = ax.imshow(image, cmap= 'Blues' , interpolation='nearest', vmin=vmin, vmax=vmax)
        cbar = fig.colorbar(pos, ax=ax, shrink=0.7)
        cbar.set_label('Velocity in m/yr', rotation=270)
        if return_ax:
            return(ax)


    def __call__(self, event):
        print('click', event)
        self.x = event.xdata
        self.y = event.ydata
        for pnt in self.plot.get_lines():
            pnt.remove()
        plt.plot(self.x, self.y, 'ro')


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
fig_xsize = 7.5
fig_ysize = 7.5
my_plot = pixelPicker(temporal_mean, fig_xsize, fig_ysize)

**Save the selected coordinates:**

In [ ]:
if (my_plot.x == None) | (my_plot.y == None):
    sarloc = (300,350)
else:
    sarloc = (int(np.ceil(my_plot.x)), int(np.ceil(my_plot.y)))
print(sarloc)

## 3.4 Plot Glacier Velocity Time Series at Point Locations

You've picked a location of interest above. Now, let's **pick a ```[5x5]```-sized rectangle around a center pixel which we selected ...**

In [ ]:
extent = (5, 5) # choose a 5 by 5 rectangle
bounds = np.s_[:, sarloc[1]:sarloc[1]+extent[1], sarloc[0]:sarloc[0]+extent[0]]

**... and extract the time series for this small area around the selected center pixel in a memory-efficient way (needed for larger stacks):**

In [ ]:
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

subset = rasterstack_masked[bounds]
aggregated = np.nanmean(subset,axis=(1,2))

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
plt.title('Glacier Velocity Results', fontsize=16)
ax.scatter(time_index[~aggregated.mask], aggregated[~aggregated.mask], color='k', marker='o')
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Glacier Velocity [m/day]', fontsize=14)
plt.xticks(rotation = 45)
ax.set_ylim([0, 4500])
plt.grid()
fig.tight_layout()